In [2]:
%reload_ext autoreload
%autoreload 2

import os
import pprint
import shutil
import pymongo

from climatools.cliradlw import (setup, pipeline)
import climatools.cliradlw.runrecord as runrecord_clirad
import climatools.cliradlw.utils as utils_clirad

from climatools.lblnew import setup_overlap
from climatools.lblnew import setup_bestfit
import climatools.lblnew.pipeline as pipeline_lblnew
import climatools.lblnew.bestfit_params as bestfit
import climatools.lblnew.runrecord_bestfit as runrecord_bestfit
import climatools.lblnew.runrecord_overlap as runrecord_overlap

In [93]:
client = pymongo.MongoClient('localhost', 27017)

In [94]:
client.cliradnew.lw.count()

234

In [75]:
ATMPROS = [('mls', 294), ('saw', 257), ('trp', 300)]

In [428]:
# EDIT HERE TO START

def params_clirad():
    band = 11
    
    from climatools.atm.absorbers import nongreys_byband
    atmpros = [('mls', 294), ('saw', 257), ('trp', 300)]
    commitnumber = '1013f91'
    molconc = nongreys_byband()[band]
    
    params = []

    # Individual (molecule, conc)
#    params.extend([{
#        'commitnumber': commitnumber,
#        'band': [band],
#        'molecule': {mol: conc},
#        'atmpro': atmpro,
#        'tsfc': tsfc} 
#        for mol, conc in molconc.items() 
#        for atmpro, tsfc in atmpros])
    
    # Overlap cases {(molecule, conc)}
    if len(molconc) > 1:
        params.extend([
            {'commitnumber': commitnumber, 
             'band': [band], 
             'molecule': molconc, 
             'atmpro': atmpro, 
             'tsfc': tsfc} for atmpro, tsfc in atmpros])
    return params

params_clirad()

[{'atmpro': 'mls',
  'band': [11],
  'commitnumber': '1013f91',
  'molecule': {'co2': 0.0004, 'h2o': 'atmpro'},
  'tsfc': 294},
 {'atmpro': 'saw',
  'band': [11],
  'commitnumber': '1013f91',
  'molecule': {'co2': 0.0004, 'h2o': 'atmpro'},
  'tsfc': 257},
 {'atmpro': 'trp',
  'band': [11],
  'commitnumber': '1013f91',
  'molecule': {'co2': 0.0004, 'h2o': 'atmpro'},
  'tsfc': 300}]

In [429]:
def to_lblnewparam(param=None):
    '''
    Convert clirad's input parameter dictionary 
    into something like lblnew's.
    '''
    band = param['band'][0]
    band = utils_clirad.mapband_new2old()[band]
    atmpro = param['atmpro']
    tsfc = param['tsfc']
    
    p = {'band': band, 
         'atmpro': atmpro, 'tsfc': tsfc,
         'dv': .001, 'nv': 1000}
    
    if len(param['molecule']) > 1: p['molecule'] = param['molecule']
    else: 
        molecule, conc = list(param['molecule'].items())[0]
        p['molecule'] = molecule
        if conc == 'atmpro': p['conc'] = None
        else: p['conc'] = conc
    return p

def make_query(param=None):
    q = {}
    for n, v in param.items():
        if n == 'molecule' and type(v) == dict:
            for mol, conc in v.items():
                q[f'param.molecule.{mol}'] = conc
        else:
            q[f'param.{n}'] = v
    return q

def get_parampairs():
    pairs = []
    for p in params_clirad():
        q_clirad = make_query(param=p)
        r_clirad = client.cliradnew.lw.count(q_clirad)
        if not r_clirad: 
            print('Following clirad case not available:')
            pprint.pprint(p)
            print('Skipping it.\n')
            continue
            
        if len(p['molecule']) > 1: collection = client.lblnew.overlap_lw
        else: collection = client.lblnew.bestfit_lw
        
        p_lblnew = to_lblnewparam(param=p)
        q_lblnew = make_query(param=p_lblnew)
        r_lblnew = collection.find_one(q_lblnew, projection=['param'])
        if not r_lblnew:
            print('Following lblnew case not available:')
            pprint.pprint(p_lblnew)
            print('Skipping it.\n')
            continue
            
        pairs.append((p, r_lblnew.get('param')))
    return pairs

In [430]:
def script_clirad_lblnew():
    '''
    Script to run clirad and then plot results against the 
    corresponding lblnew case.  This needs both input parameter
    dictionaries from clirad and lblnew.
    '''
    ps = get_parampairs()
    gprocs = pipeline.pipeline_ipynb2git(parampairs=ps, setup=setup)

In [431]:
script_clirad_lblnew()

{'atmpro': 'mls',
 'band': [11],
 'commitnumber': '1013f91',
 'molecule': {'co2': 0.0004, 'h2o': 'atmpro'},
 'tsfc': 294}
{'atmpro': 'saw',
 'band': [11],
 'commitnumber': '1013f91',
 'molecule': {'co2': 0.0004, 'h2o': 'atmpro'},
 'tsfc': 257}
{'atmpro': 'trp',
 'band': [11],
 'commitnumber': '1013f91',
 'molecule': {'co2': 0.0004, 'h2o': 'atmpro'},
 'tsfc': 300}
{'atmpro': 'mls',
 'band': [11],
 'commitnumber': '1013f91',
 'molecule': {'co2': 0.0004, 'h2o': 'atmpro'},
 'tsfc': 294}
{'atmpro': 'saw',
 'band': [11],
 'commitnumber': '1013f91',
 'molecule': {'co2': 0.0004, 'h2o': 'atmpro'},
 'tsfc': 257}
{'atmpro': 'trp',
 'band': [11],
 'commitnumber': '1013f91',
 'molecule': {'co2': 0.0004, 'h2o': 'atmpro'},
 'tsfc': 300}



In [432]:
for param, _ in get_parampairs():
    print(pipeline.nbviewer_url(param=param, setup=setup))
    print()

http://nbviewer.jupyter.org/github/qap/analysis_-_new_kdist_param/blob/master/clirad/h2o_mls_co2_0.0004/band_11/atmpro_mls/cliradlw_1013f91/results_cliradlw.ipynb

http://nbviewer.jupyter.org/github/qap/analysis_-_new_kdist_param/blob/master/clirad/h2o_saw_co2_0.0004/band_11/atmpro_saw/cliradlw_1013f91/results_cliradlw.ipynb

http://nbviewer.jupyter.org/github/qap/analysis_-_new_kdist_param/blob/master/clirad/h2o_trp_co2_0.0004/band_11/atmpro_trp/cliradlw_1013f91/results_cliradlw.ipynb

